In [1]:
import numpy as np
import sys
import pandas as pd
sys.path.insert(0, "/Users/mallen2/OpenPATH_Data/e-mission-server")

import emission.core.get_database as edb
Stage_analysis_timeseries = edb.get_analysis_timeseries_db()
Error_Rates = edb.get_Error_Rates_db()

import emission.storage.timeseries.aggregate_timeseries as estag 
agts = estag.AggregateTimeSeries()   

inferred_trips_query = Stage_analysis_timeseries.find({
    "$and": [ {"metadata.key":"analysis/confirmed_trip"},
            {"data.algorithm_chosen": {"$in": ["label_assist","sensing"]}}]
    }) 

inferred_trips_df = agts.to_data_df("analysis/confirmed_trip",inferred_trips_query)

LABEL_CATEGORIES = ['mode_confirm','purpose_confirm','replaced_mode']

intensity_df = pd.read_csv("/Users/mallen2/OpenPATH_Data/em-public-dashboard/viz_scripts/auxiliary_files/energy_intensity.csv")

storage not configured, falling back to sample, default configuration
URL not formatted, defaulting to "Stage_database"
Connecting to database URL localhost


In [2]:
def make_rel_errors(max_size):
    upper= np.random.rand(1)*max_size
    lower = np.random.rand(1)*max_size
    return  [float(lower),float(upper)]

Error_Rates = edb.get_Error_Rates_db()
# mode count intervals, (purpose count intervals later), carbon intervals, energy intervals
# Maybe eventually provide the bootstrap distributions?
modes = ['Walk', 'Bike', 'e-bike', 'Scooter share', 'Drove Alone', 'Shared Ride', 'Taxi/Uber/Lyft', 'Bus', 'Train', 'Free Shuttle', 'Air', 'Other','Total']
mode_rel_errors = {}
carbon_rel_errors = {}
energy_rel_errors = {}
for m in modes:
    if m is not 'Total':
        mode_rel_errors[m] = make_rel_errors(0.15)
    carbon_rel_errors[m] = make_rel_errors(0.2)
    energy_rel_errors[m] = make_rel_errors(0.2)

error_dictionary = {}
error_dictionary["counts"] = {}
error_dictionary["counts"]["mode_confirm"] = mode_rel_errors
error_dictionary["carbon"] = carbon_rel_errors
error_dictionary["energy"] = energy_rel_errors
error_dictionary["distance"] = {"interval": make_rel_errors(0.05), "mean": 0.01, "variance":0.02}

# to reset: db.Error_Rates.drop()

In [3]:
# Run this once. If you want to change it, drop the collection and rerun the above cell.
#Error_Rates.insert_one(error_dictionary)

### Compute Metrics with Error Bars
The following functions compute metrics and error bars using inferred trip labels. These can be added to the values calculated from trips that have user input.

In [3]:
import count_functions as counts

db_errors = Error_Rates.find_one()
rel_count_errors = db_errors["counts"]
all_inferred_counts = counts.get_inferred_counts(inferred_trips_df, LABEL_CATEGORIES)
count_intervals = counts.get_count_intervals(all_inferred_counts,LABEL_CATEGORIES, rel_count_errors)

In [4]:
inferred_trips_df['sensed_mode']

0                                     {'bicycling': 1.0}
1                                     {'bicycling': 1.0}
2      {'car': 0.641765097302607, 'walking': 0.358234...
3      {'car': 0.7295529513320035, 'walking': 0.27044...
4                                           {'car': 1.0}
                             ...                        
168                                         {'car': 1.0}
169                                     {'walking': 1.0}
170                                     {'walking': 1.0}
171                                     {'walking': 1.0}
172    {'car': 0.8208509643046544, 'walking': 0.17914...
Name: sensed_mode, Length: 173, dtype: object

In [7]:
import footprint_metrics as fm

carbon_rel_errors = db_errors["carbon"]
energy_rel_errors = db_errors["energy"]
distances_by_mode = fm.get_distances_by_mode(inferred_trips_df)
carbon_by_mode = fm.get_inferred_carbon_intervals(distances_by_mode, carbon_rel_errors,intensity_df)
energy_by_mode = fm.get_inferred_energy_intervals(distances_by_mode, energy_rel_errors,intensity_df)

# are the distances in meters in the database?

In [8]:
energy_by_mode

{'Drove Alone': {'estimate': 2592087647.3392005,
  'interval': [2668672292.462309, 3103583055.347957]},
 'Walk': {'estimate': 0.0, 'interval': [0.0, 0.0]},
 'Bike': {'estimate': 0.0, 'interval': [0.0, 0.0]}}

In [9]:
distances_by_mode

{'Other': 576713.500697305,
 'Drove Alone': 501370.91824742756,
 'Walk': 95361.01732049005,
 'Bike': 14852.071173176655,
 'Total': 1188297.5074383991}

In [10]:
intensity_df #[intensity_df["mode"] == "Car, drove alone"]["CO2_factor"]

,mode,fuel,(kWH)/trip,EI(kWH/PMT),energy_intensity_factor,energy_intensity_units,CO2_factor,CO2_factor_units
0,"Car, drove alone",gasoline,0.0000,NaN,5170.000,BTU/PMT,157.2,lb_CO2/MMBTU
1,"Car, with others",gasoline,0.0000,NaN,2585.000,BTU/PMT,157.2,lb_CO2/MMBTU
2,Taxi/Uber/Lyft,gasoline,0.0000,NaN,7214.000,BTU/PMT,157.2,lb_CO2/MMBTU
3,Bus,diesel,0.0000,NaN,4560.000,BTU/PMT,161.3,lb_CO2/MMBTU
4,Free Shuttle,diesel,0.0000,NaN,4560.000,BTU/PMT,161.3,lb_CO2/MMBTU
5,Train,electric,0.0000,0.370,0.370,kWH/PMT,1166.0,lb_CO2/MWH
6,Scooter share,electric,0.0041,0.027,0.027,kWH/PMT,1166.0,lb_CO2/MWH
7,Pilot ebike,electric,0.0000,0.022,0.022,kWH/PMT,1166.0,lb_CO2/MWH
8,Bikeshare,human_powered,0.0900,0.000,0.000,kWH/PMT,1166.0,lb_CO2/MWH
9,Walk,human_powered,0.0000,NaN,0.000,NaN,0.0,0
